In [1]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml
from scipy.optimize import fmin_l_bfgs_b


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
Data, labels = ml.loadFile('./Train.txt')
n, m = sum(labels == 1), sum(labels == 0)
print('Prima ', n/m)
numpy.random.seed(0)
indexes = numpy.random.permutation(1200)
Data_s = Data[:, indexes]
label_s = labels[indexes]
ns, ms = sum(label_s == 1), sum(label_s == 0)
print('Dopo :', ns/ms)

Prima  0.10125801677355697
Dopo : 0.10497237569060773


In [3]:
(DTR, LTR), (DTE, LTE) = ml.split_db_2to1(Data_s, label_s, seed=42)
n1, n0 = sum(LTR == 1), sum(LTR == 0)
print('Dopo dopo', n1/n0)


Dopo dopo 0.11265646731571627


In [4]:
def L_dual_wrapper(H_hat):
    def L_dual(alpha):
        one_vect = numpy.ones((len(alpha)), dtype='int32')
        L_d = 1/2 * numpy.dot( alpha.T, numpy.dot( H_hat, alpha ) ) - numpy.dot( alpha.T, one_vect )
        grad_L_d = numpy.dot( H_hat, alpha ) - one_vect

        v = numpy.array((L_d, grad_L_d), dtype=object)

        return v
    return L_dual

def linearSVM(DTR, LTR, DTE, params):
    K, C = params[0], params[1]
    x0 = numpy.zeros((DTR.shape[1]), dtype='int32')
    D_hat = numpy.vstack([DTR, K*numpy.ones((DTR.shape[1]))])
    H_hat = ml.compute_H_hat(D_hat, LTR)
    boundaries = []
    for i in range(DTR.shape[1]):
        boundaries.append((0, C))
        
    alpha, f, d = fmin_l_bfgs_b(L_dual_wrapper(H_hat), x0, bounds=boundaries, factr=1.0)
    w = ml.compute_w(D_hat, LTR, alpha)

    DTE_hat = numpy.vstack([DTE, K * numpy.ones((DTE.shape[1]))])
    
    s = list(numpy.dot(w, DTE_hat))
        
    
    return s


In [5]:
def kfold(D, L, fold, params, app):
    error = 0
    N = int(D.shape[1]/fold) #numero di elementi per ogni fold
    numpy.random.seed(0) #imposto il seed del generatore di numeri casuali -> in tal modo genererò la stessa sequenza di numeri casuali aventi seed uguale
    indexes = numpy.random.permutation(D.shape[1]) #genero una sequenza di numeri casuali che vanno da 0 al num_di_campioni
    
    LTE_final = []
    llr_final = []
    for j in range(fold):
        test_indexes = indexes[(j*N):((j+1)*N)] #selezioni gli indici che identificano i campioni (casuali) del test set
        if(j > 0): #se il test set non è preso dalla prima fold (--> il test set è una fold intermedia o l'ultima fold)
            left_indexes = indexes[0:(j*N)] #allora prendo tutti gli indici che stanno a sinistra di tale fold
        else: #se il test set è preso dalla prima fold
            right_indexes = indexes[((j+1)*N):] #prendo tutti gli indici a destra della prima fold

        if(j == 0): #se il test set è preso dalla prima fold
            train_indexes = right_indexes #assegno agli indici di training quelli che stanno a destra della prima fold
        elif(j == fold-1): #se il test set è preso dall'ultima fold
            train_indexes = left_indexes #assegno agli indici di training quelli che stanno a sinistra dell'ultima fold
        else: #in questo caso il test set è preso da una fold intermedia
            train_indexes = numpy.hstack((left_indexes, right_indexes)) #pertanto assegno agli indici di training quelli appartenenti alle fold di sinistra e di destra

        DTR = D[:, train_indexes]  #definisco insieme di training e di testing
        LTR = L[train_indexes]
        DTE = D[:, test_indexes]
        LTE = L[test_indexes]
        LTE_final.extend(LTE)
        llr_final.extend(linearSVM(DTR, LTR, DTE, params))
        
    CM = ml.compute_optimal_B_decision(app, llr_final, LTE_final)
    sensitivity = (1-ml.compute_FNR(CM)) # TPR
    specificity = (1-ml.compute_FPR(CM)) # TNR
        
    app_bayes_risk=ml.compute_Bayes_risk(CM, app)
    DCF = ml.compute_norm_Bayes(app_bayes_risk, app)

    minDCF= ml.compute_min_DCF(llr_final, app, LTE_final)
    error = 1-(CM[0, 0]+CM[1,1])/(len(LTE_final))

    print("\-/ \-/ \-/ \-/ \-/ ")
    print("Linear SVM error:", error)
    print(app,"DCF:", DCF, "minDCF:", minDCF)
    print('Sensitivity (TPR):', sensitivity, ' Specificity (TNR): ', specificity)
    print('CM\n', CM)
    print("/-\ /-\ /-\ /-\ /-\ ")

In [7]:
kfold(Data, labels, 5, [1, 0.1], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.026106442577030764
[0.5, 1, 1] DCF: 0.22469017270008768 minDCF: 0.14696761828266638
Sensitivity (TPR): 0.781973203410475  Specificity (TNR):  0.9933366238894373
CM
 [[8050.  179.]
 [  54.  642.]]
/-\ /-\ /-\ /-\ /-\ 


In [8]:
kfold(Data, labels, 5, [1, 1], [0.5, 1, 1])
kfold(Data, labels, 5, [1, 10], [0.5, 1, 1])
kfold(Data, labels, 5, [10, 0.1], [0.5, 1, 1])
kfold(Data, labels, 5, [10, 1], [0.5, 1, 1])
kfold(Data, labels, 5, [10, 10], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.6024649859943978
[0.5, 1, 1] DCF: 0.9065075756938124 minDCF: 0.7398214201976017
Sensitivity (TPR): 0.7295980511571254  Specificity (TNR):  0.36389437314906214
CM
 [[2949.  222.]
 [5155.  599.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.6283473389355743
[0.5, 1, 1] DCF: 0.8715234232685202 minDCF: 0.808734773162048
Sensitivity (TPR): 0.8002436053593179  Specificity (TNR):  0.3282329713721619
CM
 [[2660.  164.]
 [5444.  657.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.03742296918767507
[0.5, 1, 1] DCF: 0.19446254717899944 minDCF: 0.18359589646411512
Sensitivity (TPR): 0.8294762484774665  Specificity (TNR):  0.976061204343534
CM
 [[7910.  140.]
 [ 194.  681.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.6066106442577031
[0.5, 1, 1] DCF: 0.831165163471701 minDCF: 0.8217690426405571
Sensitivity (TPR): 0.8185140073081608  Specificity (TNR):  0.3503208292201382
CM
 [[2839.  149.]
 [5265.  672.]]


In [9]:
kfold(Data, labels, 5, [1, 0.01], [0.5, 1, 1])
kfold(Data, labels, 5, [10, 0.01], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.025322128851540615
[0.5, 1, 1] DCF: 0.24352975869121637 minDCF: 0.1317024539692884
Sensitivity (TPR): 0.7600487210718636  Specificity (TNR):  0.99642152023692
CM
 [[8075.  197.]
 [  29.  624.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.02554621848739491
[0.5, 1, 1] DCF: 0.24596581228439543 minDCF: 0.12369359712290767
Sensitivity (TPR): 0.7576126674786845  Specificity (TNR):  0.99642152023692
CM
 [[8075.  199.]
 [  29.  622.]]
/-\ /-\ /-\ /-\ /-\ 


In [11]:
data_g = ml.gaussianize(Data)
data_z = ml.z_normalization(Data)

In [12]:
kfold(data_g, labels, 5, [1, 0.1], [0.5, 1, 1])
kfold(data_g, labels, 5, [1, 1], [0.5, 1, 1])
kfold(data_g, labels, 5, [1, 10], [0.5, 1, 1])
kfold(data_g, labels, 5, [10, 0.1], [0.5, 1, 1])
kfold(data_g, labels, 5, [10, 1], [0.5, 1, 1])
kfold(data_g, labels, 5, [10, 10], [0.5, 1, 1])


\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.022969187675070057
[0.5, 1, 1] DCF: 0.16978743448446684 minDCF: 0.13403885902271687
Sensitivity (TPR): 0.8392204628501827  Specificity (TNR):  0.9909921026653504
CM
 [[8031.  132.]
 [  73.  689.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.021848739495798353
[0.5, 1, 1] DCF: 0.15651253557588138 minDCF: 0.12745514162417199
Sensitivity (TPR): 0.8526187576126675  Specificity (TNR):  0.9908687068114511
CM
 [[8030.  121.]
 [  74.  700.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.021624649859943945
[0.5, 1, 1] DCF: 0.1518872200973219 minDCF: 0.12768600158956706
Sensitivity (TPR): 0.8574908647990256  Specificity (TNR):  0.9906219151036525
CM
 [[8028.  117.]
 [  76.  704.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.022521008403361353
[0.5, 1, 1] DCF: 0.1671045891834892 minDCF: 0.13365273971861535
Sensitivity (TPR): 0.8416565164433618  Specificity (TNR):  0.991238894373149
CM
 [[8033.  130.]
 [  

In [13]:
kfold(data_z, labels, 5, [1, 0.1], [0.5, 1, 1])
kfold(data_z, labels, 5, [1, 1], [0.5, 1, 1])
kfold(data_z, labels, 5, [1, 10], [0.5, 1, 1])
kfold(data_z, labels, 5, [10, 0.1], [0.5, 1, 1])
kfold(data_z, labels, 5, [10, 1], [0.5, 1, 1])
kfold(data_z, labels, 5, [10, 10], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.02285714285714291
[0.5, 1, 1] DCF: 0.20797612162472512 minDCF: 0.10940372598364982
Sensitivity (TPR): 0.7965895249695494  Specificity (TNR):  0.9954343534057256
CM
 [[8067.  167.]
 [  37.  654.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.021848739495798353
[0.5, 1, 1] DCF: 0.1926353566846585 minDCF: 0.11083668100323084
Sensitivity (TPR): 0.8124238733252132  Specificity (TNR):  0.9949407699901284
CM
 [[8063.  154.]
 [  41.  667.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.021512605042016797
[0.5, 1, 1] DCF: 0.19007590723758014 minDCF: 0.11058988929543223
Sensitivity (TPR): 0.8148599269183923  Specificity (TNR):  0.9950641658440277
CM
 [[8064.  152.]
 [  40.  669.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Linear SVM error: 0.02285714285714291
[0.5, 1, 1] DCF: 0.20797612162472512 minDCF: 0.10917286601825477
Sensitivity (TPR): 0.7965895249695494  Specificity (TNR):  0.9954343534057256
CM
 [[8067.  167.]
 [  